In [2]:
!pip install selenium

  Using cached trio-0.29.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached trio_websocket-0.12.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached cffi-1.17.1-cp312-cp312-win_amd64.whl.metadata (1.6 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl.metadata (13 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   ---------------------------------------- 0.1/9.4 MB 1.7 MB/s eta 0:00:06
   - -------------------------------------- 0.2/9.4 MB 3.0 MB/s eta 0:00:04
   -- -------------------------


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!pip install webdriver_manager


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [113]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import os
import requests

In [179]:
from selenium.webdriver.chrome.options import Options

options = Options()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36")

In [180]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Ejecuta JavaScript para redefinir navigator.webdriver
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
    "source": """
    Object.defineProperty(navigator, 'webdriver', {
        get: () => undefined
    })
    """
})


{'identifier': '2'}

In [163]:
# Configurar el servicio de ChromeDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

In [181]:
url = "https://www.vogue.com/fashion-shows/image-archive"
driver.get(url)

In [182]:
# Aceptamos las cookies

try:
    accept_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="onetrust-accept-btn-handler"]'))
    )
    accept_button.click()
    print("Cookies accepted.")
except Exception as e:
    print("Couldn't find such button:", e)

Cookies accepted.


In [183]:
# Vamos a la página de registro

try:
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable(
    (By.XPATH, '//*[@id="app-root"]/div/div/div[6]/div[1]/div/div/div/div[1]/div/div/a')
    )).click()
    print("Accediendo al formulario de inicio de sesión...")
except Exception as e:
    print("Couldn't find such button:", e)

Accediendo al formulario de inicio de sesión...


In [184]:
# Aceptamos cookies de nuevo

try:
    accept_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="onetrust-accept-btn-handler"]'))
    )
    accept_button.click()
    print("Cookies accepted.")
except Exception as e:
    print("Couldn't find such button:", e)

Cookies accepted.


In [185]:
# Credenciales

email = "patrialcalde03@gmail.com"
password = "alexander_mcqueen_2008"

In [186]:
try:

    # email
    email_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "TextField-id-email"))).send_keys(email)
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "email-continue-button"))).click()
    print("Email introduced")

    # password
    password_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="TextField-id-password"]'))).send_keys(password)
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="log-in-sign-in-button-password"]/span'))).click()
    print('Signed in successfully')

except Exception as e:
    print("Sign in failed:", e)

Email introduced
Signed in successfully


In [ ]:
image_limit = 100  # Cámbialo a lo que quieras

# Asegurarse de que la carpeta existe
os.makedirs("fashion_images_2", exist_ok=True)

# Esperar a que las imágenes estén presentes en el DOM
WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((
    By.XPATH, '//*[@id="main-content"]/div/div[3]/div[1]/div/div'
)))

# Obtener las imágenes
images = driver.find_elements(By.XPATH, '//*[@id="main-content"]/div/div[3]/div[1]/div/div')

while len(images) < image_limit:
    print('Not enough images on this page. Loading more...') 
    view_more_button = driver.find_element(By.XPATH, '//*[@id="main-content"]/div/div[3]/button/span')
    view_more_button.click()
    images = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((
    By.XPATH, '//*[@id="main-content"]/div/div[3]/div[1]/div/div'
)))

print(len(images))

downloaded = 0
pic_n = 0

for img in images[0:image_limit]:
    # print(img)
    try:
        picture = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, f'//*[@id="main-content"]/div/div[3]/div[1]/div/div[{pic_n+1}]/span[1]/picture/img'))
        )
        # print(picture)
        src =  picture.get_attribute("src") or picture.get_attribute("data-src")
        # print(src)
        if not src or ".jpg" not in src:
            print("URL inválida, saltando...")
            continue

        filename = f'image_{downloaded}.png'

        # Ruta de guardado
        path = os.path.join("fashion_images", filename)

        # Descargar y guardar
        response = requests.get(src)
        if response.status_code == 200:
            with open(path, "wb") as f:
                f.write(response.content)
            print(f"✅ Imagen guardada: {filename}")
            downloaded += 1
        else:
            print(f"❌ No se pudo descargar {src}")

    except Exception as e:
        print(f"⚠️ Error con una imagen: {e}")

    pic_n += 1

print(f"\n🔚 Descarga completa: {downloaded} imágenes.")